In [1]:
import numpy as np
from scipy.stats import multivariate_normal
from matplotlib import pyplot as plt
from copy import copy
from datetime import datetime, timedelta

from stonesoup.functions import gm_reduce_single

from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.updater.kalman import KalmanUpdater
from stonesoup.predictor.imm import IMMPredictor
from stonesoup.updater.imm import IMMUpdater
from stonesoup.models.transition.linear import ConstantVelocity, \
    CombinedLinearGaussianTransitionModel, LinearGaussianTimeInvariantTransitionModel

from stonesoup.models.measurement.linear import LinearGaussian

from stonesoup.types.state import GaussianState
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.detection import Detection
from stonesoup.types.state import StateVector, CovarianceMatrix, \
    GaussianMixtureState, WeightedGaussianState
from stonesoup.types.track import Track

from stonesoup.simulator.simple import SingleTargetGroundTruthSimulator

from matplotlib.patches import Ellipse


In [2]:
from emgpb2.states import Gaussian as EMGPB2Gaussian
from emgpb2.states import GMM as EMGPB2GMM
from emgpb2.states import GMMSequence as EMGPB2GMMSequence
from emgpb2.models import LinearModel as EMGPB2LinearModel
from emgpb2.models import ConstantVelocity as EMGPB2ConstantVelocity
from emgpb2.EM import SKFEstimator

Simulate groundtruth and detections

In [3]:
# dimension of state vector and observation
state_dim = 4
obs_dim = 2

# define transition models
gt_transition_model_1 = CombinedLinearGaussianTransitionModel(
                        (ConstantVelocity(1.0),
                         ConstantVelocity(1.0)))
gt_transition_model_2 = CombinedLinearGaussianTransitionModel(
                        (ConstantVelocity(8.0),
                         ConstantVelocity(8.0)))

# define measurement models
mesurement_noise_1 = np.diag([0.5 ** 2, 0.5 ** 2])
measurement_model_1 = LinearGaussian(ndim_state=4, mapping=[0, 2], noise_covar=mesurement_noise_1)
mesurement_noise_2 = np.diag([0.25 ** 2, 0.25 ** 2])
measurement_model_2 = LinearGaussian(ndim_state=4, mapping=[0, 2], noise_covar=mesurement_noise_2)

# iteration numbers of two models
iter_model_1 = 220
iter_model_2 = 180

# generate groundtruth
gt_track = []
gt_time = []
timestamp_init = datetime.now()

# model 1
state_init_1 = GaussianState(StateVector(np.zeros((state_dim, 1))),
                           CovarianceMatrix(np.zeros((state_dim, state_dim))),
                           timestamp=timestamp_init)
gt_generator_1 = SingleTargetGroundTruthSimulator(gt_transition_model_1, state_init_1, number_steps=iter_model_1)
for time, gnd_paths in gt_generator_1.groundtruth_paths_gen():
    gnd_path = gnd_paths.pop()
    gt_track.append(gnd_path.state)
    gt_time.append(time)

# model 2
state_init_2 = GaussianState(gt_track[-1].state_vector, 
                             CovarianceMatrix(np.zeros((state_dim, state_dim))), 
                             timestamp=gt_time[-1])
gt_generator_2 = SingleTargetGroundTruthSimulator(gt_transition_model_2, state_init_2, number_steps=iter_model_2)
for time, gnd_paths in gt_generator_2.groundtruth_paths_gen():
    gnd_path = gnd_paths.pop()
    gt_track.append(gnd_path.state)
    gt_time.append(time)

# generate detections (adding noise)
# model 1
detection_track = []
for i in range(iter_model_1):
    m_ = Detection(measurement_model_1.function(gt_track[i].state_vector, measurement_model_1.rvs(1)), gt_time[i])
    detection_track.append(m_)
# model 2
for i in range(iter_model_2):
    ii = i + iter_model_1
    m_ = Detection(measurement_model_2.function(gt_track[ii].state_vector, measurement_model_2.rvs(1)), gt_time[ii])
    detection_track.append(m_)

Filter model. Testing by correct parameters.

In [4]:
# model transition probability
model_transition_matrix = np.array([[0.5, 0.5],
                                    [0.5, 0.5]])

# define two kalman predictors. re-use the groundtruth transition model.
predictor_1 = KalmanPredictor(gt_transition_model_1)
predictor_2 = KalmanPredictor(gt_transition_model_2)

# define IMM predictor
imm_predictor = IMMPredictor([predictor_1, predictor_2],
                             model_transition_matrix)

# define two kalman updaters. re-use the groundtruth measurement model.
updater_1 = KalmanUpdater(measurement_model_1)
updater_2 = KalmanUpdater(measurement_model_2)

# define IMM updater
imm_updater = IMMUpdater([updater_1, updater_2], model_transition_matrix)

# initialise state for kalman filter 1
ft_state_init_1 = WeightedGaussianState(StateVector(np.zeros((state_dim, 1))),
                                      CovarianceMatrix(np.eye(state_dim) * 1.0),
                                      timestamp=timestamp_init,
                                      weight=0.5)

# initialise state for kalman filter 2
ft_state_init_2 = WeightedGaussianState(StateVector(np.zeros((state_dim, 1))),
                                      CovarianceMatrix(np.eye(state_dim) * 1.0),
                                      timestamp=timestamp_init,
                                      weight=0.5)

# define Gaussian Mixture model for initial state
prior = GaussianMixtureState([ft_state_init_1, ft_state_init_2])
track = Track([copy(prior)])

# tracking
track_error = []
for i in range(1, len(detection_track)-1):
    # measurement
    measurement = detection_track[i]
    # State prediction
    prediction = imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = imm_updater.predict_measurement(prediction)
    # Association
    hyp = SingleHypothesis(prediction, measurement)
    # State update
    prior = imm_updater.update(hyp)
    track.append(prior)
    track_error.append(measurement_model_1.function(track[i].state_vector, np.zeros((obs_dim, 1)))
                       - measurement_model_1.function(gt_track[i].state_vector, np.zeros((obs_dim, 1))))

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 0.5758576894330582


Gussing some IMM paramters.

In [5]:
init_P = [5.0, 5.0]
q = [1.0, 15.0]
r = [3.0, 3.0]

Filter model. Testing by guessing IMM parameters.

In [6]:
# define model transition probability
ft_model_transition_matrix = np.array([[0.5, 0.5],
                                       [0.5, 0.5]])
# define transition model 1 using guessed parameter
ft_transition_model_1 = CombinedLinearGaussianTransitionModel(
                        (ConstantVelocity(q[0]),
                         ConstantVelocity(q[0])))
# define transition model 2 using guessed parameter
ft_transition_model_2 = CombinedLinearGaussianTransitionModel(
                        (ConstantVelocity(q[1]),
                         ConstantVelocity(q[1])))

# define measurement model. use same measurement model for two Kalman filters.
ft_measurement_noise = np.diag([r[0] ** 2, r[0] ** 2])
ft_measurement_model = LinearGaussian(ndim_state=4, mapping=[0, 2],
                                   noise_covar=ft_measurement_noise)

# define two kalman updaters.
ft_predictor_1 = KalmanPredictor(ft_transition_model_1)
ft_predictor_2 = KalmanPredictor(ft_transition_model_2)

# define IMM predictor
ft_imm_predictor = IMMPredictor([ft_predictor_1, ft_predictor_2], ft_model_transition_matrix)

# define IMM updater. use same measurement model for two Kalman filters.
ft_updater = KalmanUpdater(ft_measurement_model)
ft_imm_updater = IMMUpdater([copy(ft_updater), copy(ft_updater)], ft_model_transition_matrix)

# define Gaussian Mixture model for initial state. We used the same initial state as previous filter.
prior = GaussianMixtureState([ft_state_init_1, ft_state_init_2])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)-1):
    # measurement
    measurement = detection_track[i]
    # State prediction
    prediction = ft_imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = ft_imm_updater.predict_measurement(prediction)
    # Association
    hyp = SingleHypothesis(prediction, measurement)
    # State update
    prior = ft_imm_updater.update(hyp)
    track.append(prior)
    track_error.append(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))) - ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 0.9833021174196005


Run EMGPB2 to estimate IMM parameters.

In [7]:
num_of_models = len(q)
gaussian_models = []
for i in range(num_of_models):
    gaussian_models.append(EMGPB2Gaussian(np.zeros([state_dim, 1]), (init_P[i] ** 2) * np.eye(state_dim)))
initial_gmm_state = EMGPB2GMM(gaussian_models)
# measurement sequence
detection_track_EMGPB2 = [ele.state_vector for ele in detection_track]
gmmsequence = EMGPB2GMMSequence(np.float64(detection_track_EMGPB2), initial_gmm_state)
dataset = [gmmsequence]
# Initial models. use guessed parameters.
constantvelocity_models = []
for i in range(num_of_models):
    constantvelocity_models.append(EMGPB2ConstantVelocity(dt=1.0, q=q[i], r=r[i], 
                                                          state_dim=state_dim, obs_dim=obs_dim, mode='diag'))

# Switching matrix probability
Z = np.ones((2, 2)) / 2

models_all, Z_all, dataset, LLs = SKFEstimator.EM(dataset, constantvelocity_models, Z,
                                                 max_iters=300, threshold=1e-8, learn_H=True, learn_R=True,
                                                 learn_A=True, learn_Q=True, learn_init_state=False, learn_Z=True,
                                                 diagonal_Q=False, wishart_prior=False)


new_models = new_models_all[-1]

# new_models[0].Q = (new_models[0].Q + new_models[0].Q.T)/2
# new_models[1].Q = (new_models[1].Q + new_models[1].Q.T)/2
# new_models[0].R = (new_models[0].R + new_models[0].R.T)/2
# new_models[1].R = (new_models[1].R + new_models[1].R.T)/2

-2380.5209185206613
model -- 0 new_A: 
[[ 1.00044646e+00  9.40075465e-01  5.82323162e-04 -4.12918953e-02]
 [ 4.84635700e-04  9.32694996e-01  7.25141611e-04 -5.37382233e-02]
 [-2.42713806e-04 -2.55857204e-02  1.00027105e+00  9.40324811e-01]
 [-3.32891918e-04 -2.35971236e-02  2.15130062e-04  9.40613626e-01]]
model -- 1 new_A: 
[[ 1.00037163e+00  9.32914882e-01  6.29114550e-04 -5.06320099e-02]
 [ 5.20666762e-04  9.21139879e-01  8.41508979e-04 -6.33384714e-02]
 [-2.38490326e-04 -2.53300626e-02  1.00023558e+00  9.44643407e-01]
 [-3.35806202e-04 -2.62415342e-02  2.14184530e-04  9.39819418e-01]]
model -- 0 new_Q: 
[[ 5.6451178   4.71771691 -0.13697146 -0.1299568 ]
 [ 4.71771691  7.80075158 -0.14251053 -0.13491509]
 [-0.13697146 -0.14251053  5.4364837   4.47365755]
 [-0.1299568  -0.13491509  4.47365755  7.35015079]]
model -- 1 new_Q: 
[[ 6.74396534  5.78559269 -0.31646008 -0.31017165]
 [ 5.78559269  9.72658212 -0.2926058  -0.31683999]
 [-0.31646009 -0.2926058   6.49177897  5.57814419]
 [-0.310

-1764.200233543218
model -- 0 new_A: 
[[ 1.00044517e+00  9.37979969e-01  6.02148290e-04 -4.30800590e-02]
 [ 1.80499046e-04  9.68489493e-01  4.44780393e-04 -3.74575161e-02]
 [-2.52775527e-04 -2.56323891e-02  1.00026901e+00  9.39517933e-01]
 [-2.71163147e-04 -1.85018894e-03 -5.15415903e-05  9.84002763e-01]]
model -- 1 new_A: 
[[ 1.00044485e+00  9.37984322e-01  6.02070219e-04 -4.30936670e-02]
 [ 1.80204827e-04  9.68494936e-01  4.44695290e-04 -3.74680069e-02]
 [-2.52686814e-04 -2.56304819e-02  1.00026897e+00  9.39529495e-01]
 [-2.71093193e-04 -1.85001639e-03 -5.15703973e-05  9.84012250e-01]]
model -- 0 new_Q: 
[[ 2.23953833  2.03978684 -0.03392995 -0.02062039]
 [ 2.03978771  3.91282267  0.09894373  0.14173757]
 [-0.03390958  0.09893128  1.99001196  1.77063165]
 [-0.02063319  0.14167545  1.7706323   3.40130433]]
model -- 1 new_Q: 
[[ 2.23962547  2.03979137 -0.03401312 -0.02067698]
 [ 2.03979224  3.91281872  0.09890885  0.14169623]
 [-0.03399274  0.0988964   1.99011308  1.77072883]
 [-0.0206

-1708.9577955144189
model -- 0 new_A: 
[[ 1.00044515e+00  9.38235850e-01  6.00375703e-04 -4.28010125e-02]
 [ 1.81614422e-04  9.68312626e-01  4.47751805e-04 -3.77359466e-02]
 [-2.50703480e-04 -2.57212277e-02  1.00027095e+00  9.39456970e-01]
 [-2.72306462e-04 -1.14551198e-03 -5.61956188e-05  9.84655056e-01]]
model -- 1 new_A: 
[[ 1.00044508e+00  9.38236440e-01  6.00363962e-04 -4.28041749e-02]
 [ 1.81556842e-04  9.68313390e-01  4.47739395e-04 -3.77386953e-02]
 [-2.50675999e-04 -2.57211071e-02  1.00027095e+00  9.39459420e-01]
 [-2.72282271e-04 -1.14569524e-03 -5.61974785e-05  9.84657263e-01]]
model -- 0 new_Q: 
[[ 1.79084766  2.01597967 -0.027165   -0.04606092]
 [ 2.0159796   3.96232682  0.13568165  0.14461978]
 [-0.02706225  0.13573152  1.54596123  1.70641253]
 [-0.04600844  0.14470454  1.70641283  3.36186386]]
model -- 1 new_Q: 
[[ 1.7908632   2.01597731 -0.02718262 -0.04607373]
 [ 2.01597724  3.96232176  0.13567353  0.14460729]
 [-0.02707987  0.13572339  1.54598009  1.70643153]
 [-0.046

-1703.3910982605607
model -- 0 new_A: 
[[ 1.00044803e+00  9.37894855e-01  6.03438573e-04 -4.29916989e-02]
 [ 1.97676566e-04  9.66196669e-01  4.66185409e-04 -3.90219426e-02]
 [-2.50027865e-04 -2.59524136e-02  1.00027399e+00  9.39095776e-01]
 [-2.74461417e-04 -2.08370302e-03 -4.38443966e-05  9.82734215e-01]]
model -- 1 new_A: 
[[ 1.00044800e+00  9.37895151e-01  6.03432761e-04 -4.29933281e-02]
 [ 1.97646107e-04  9.66197049e-01  4.66179222e-04 -3.90234672e-02]
 [-2.50012928e-04 -2.59523938e-02  1.00027399e+00  9.39097003e-01]
 [-2.74447498e-04 -2.08382243e-03 -4.38446718e-05  9.82735377e-01]]
model -- 0 new_Q: 
[[ 1.74774126  2.13538881 -0.02354036 -0.04727658]
 [ 2.13538814  4.22026144  0.15293134  0.16049984]
 [-0.02342896  0.152994    1.50534921  1.80621853]
 [-0.04721048  0.1606212   1.80621893  3.57658026]]
model -- 1 new_Q: 
[[ 1.74774682  2.13538571 -0.02354811 -0.04728235]
 [ 2.13538504  4.2202564   0.15292786  0.16049362]
 [-0.02343669  0.15299052  1.50535771  1.80622746]
 [-0.047

-1702.801037965723
model -- 0 new_A: 
[[ 1.00045028e+00  9.37608687e-01  6.05932891e-04 -4.31593808e-02]
 [ 2.04743543e-04  9.65250265e-01  4.74423464e-04 -3.96075155e-02]
 [-2.50006110e-04 -2.61192625e-02  1.00027607e+00  9.38805358e-01]
 [-2.74811814e-04 -2.59613671e-03 -3.76278350e-05  9.81823529e-01]]
model -- 1 new_A: 
[[ 1.00045026e+00  9.37608866e-01  6.05929447e-04 -4.31603133e-02]
 [ 2.04725400e-04  9.65250496e-01  4.74419729e-04 -3.96084150e-02]
 [-2.49997535e-04 -2.61192528e-02  1.00027607e+00  9.38806050e-01]
 [-2.74803698e-04 -2.59620255e-03 -3.76279069e-05  9.81824194e-01]]
model -- 0 new_Q: 
[[ 1.74543048  2.18907194 -0.02169785 -0.04198927]
 [ 2.18907111  4.33454167  0.16052657  0.17380914]
 [-0.02159246  0.16058714  1.50409139  1.85255472]
 [-0.04192529  0.17392749  1.85255522  3.6734982 ]]
model -- 1 new_Q: 
[[ 1.74543289  2.18906946 -0.0217018  -0.04199218]
 [ 2.18906863  4.33453787  0.16052496  0.17380594]
 [-0.02159641  0.16058553  1.50409596  1.85255958]
 [-0.0419

...

-1702.7820193522352
model -- 0 new_A: 
[[ 1.00050113e+00  9.31535875e-01  6.56094370e-04 -4.60958696e-02]
 [ 2.00612370e-04  9.65500434e-01  4.73165531e-04 -3.97518415e-02]
 [-2.64366559e-04 -2.88626635e-02  1.00031083e+00  9.32596218e-01]
 [-2.74585580e-04 -2.61577870e-03 -3.78522843e-05  9.81869293e-01]]
model -- 1 new_A: 
[[ 1.00050113e+00  9.31535875e-01  6.56094355e-04 -4.60958755e-02]
 [ 2.00612299e-04  9.65500435e-01  4.73165519e-04 -3.97518458e-02]
 [-2.64366495e-04 -2.88626636e-02  1.00031083e+00  9.32596223e-01]
 [-2.74585528e-04 -2.61577935e-03 -3.78522871e-05  9.81869297e-01]]
model -- 0 new_Q: 
[[ 1.8810607   2.16777585 -0.03526735 -0.02501837]
 [ 2.16777709  4.3466801   0.1870165   0.22705298]
 [-0.03514773  0.18708102  1.62561298  1.85153124]
 [-0.02494953  0.22714507  1.85153076  3.6936363 ]]
model -- 1 new_Q: 
[[ 1.88106075  2.16777586 -0.03526738 -0.02501841]
 [ 2.16777709  4.34668011  0.18701648  0.22705294]
 [-0.03514778  0.187081    1.62561305  1.85153129]
 [-0.024

-1702.783613439246
model -- 0 new_A: 
[[ 1.00050251e+00  9.31373584e-01  6.57429473e-04 -4.61723664e-02]
 [ 2.00424154e-04  9.65518101e-01  4.73034288e-04 -3.97482533e-02]
 [-2.64771414e-04 -2.89331475e-02  1.00031173e+00  9.32432333e-01]
 [-2.74592897e-04 -2.60867649e-03 -3.79435287e-05  9.81881495e-01]]
model -- 1 new_A: 
[[ 1.00050251e+00  9.31373585e-01  6.57429449e-04 -4.61723745e-02]
 [ 2.00424043e-04  9.65518102e-01  4.73034268e-04 -3.97482595e-02]
 [-2.64771330e-04 -2.89331477e-02  1.00031173e+00  9.32432340e-01]
 [-2.74592829e-04 -2.60867734e-03 -3.79435287e-05  9.81881500e-01]]
model -- 0 new_Q: 
[[ 1.88398964  2.16646978 -0.03562229 -0.02473086]
 [ 2.16647107  4.34578096  0.18763548  0.22816258]
 [-0.03550227  0.18770014  1.62823939  1.85087662]
 [-0.02466185  0.22825418  1.85087611  3.6931376 ]]
model -- 1 new_Q: 
[[ 1.8839897   2.16646977 -0.03562236 -0.0247309 ]
 [ 2.16647106  4.34578095  0.18763546  0.22816254]
 [-0.03550233  0.18770011  1.62823944  1.85087666]
 [-0.0246

-1702.785237033894
model -- 0 new_A: 
[[ 1.00050389e+00  9.31210556e-01  6.58770488e-04 -4.62491483e-02]
 [ 2.00236749e-04  9.65535671e-01  4.72903832e-04 -3.97447076e-02]
 [-2.65178551e-04 -2.90038462e-02  1.00031263e+00  9.32267809e-01]
 [-2.74600417e-04 -2.60163337e-03 -3.80340713e-05  9.81893579e-01]]
model -- 1 new_A: 
[[ 1.00050389e+00  9.31210558e-01  6.58770461e-04 -4.62491574e-02]
 [ 2.00236632e-04  9.65535672e-01  4.72903811e-04 -3.97447141e-02]
 [-2.65178456e-04 -2.90038464e-02  1.00031263e+00  9.32267816e-01]
 [-2.74600342e-04 -2.60163434e-03 -3.80340721e-05  9.81893585e-01]]
model -- 0 new_Q: 
[[ 1.88689574  2.16516439 -0.03597517 -0.02444087]
 [ 2.16516574  4.3449034   0.18825642  0.22927485]
 [-0.03585464  0.18832126  1.6308473   1.85022288]
 [-0.02437165  0.22936597  1.85022233  3.69265542]]
model -- 1 new_Q: 
[[ 1.88689583  2.1651644  -0.03597525 -0.02444092]
 [ 2.16516576  4.34490341  0.18825639  0.2292748 ]
 [-0.03585473  0.18832122  1.63084738  1.85022294]
 [-0.0243

-1702.78688941717
model -- 0 new_A: 
[[ 1.00050527e+00  9.31046797e-01  6.60117377e-04 -4.63262127e-02]
 [ 2.00050168e-04  9.65553143e-01  4.72774162e-04 -3.97412032e-02]
 [-2.65587956e-04 -2.90747577e-02  1.00031354e+00  9.32102649e-01]
 [-2.74608128e-04 -2.59464990e-03 -3.81239119e-05  9.81905545e-01]]
model -- 1 new_A: 
[[ 1.00050527e+00  9.31046799e-01  6.60117346e-04 -4.63262213e-02]
 [ 2.00050026e-04  9.65553145e-01  4.72774133e-04 -3.97412103e-02]
 [-2.65587872e-04 -2.90747575e-02  1.00031354e+00  9.32102656e-01]
 [-2.74608060e-04 -2.59465046e-03 -3.81239111e-05  9.81905551e-01]]
model -- 0 new_Q: 
[[ 1.88977898  2.16385976 -0.03632578 -0.02414838]
 [ 2.16386118  4.3440474   0.18887937  0.23038978]
 [-0.03620476  0.18894439  1.63343667  1.84957001]
 [-0.02407895  0.23048044  1.84956942  3.69218967]]
model -- 1 new_Q: 
[[ 1.88977902  2.16385974 -0.03632582 -0.02414841]
 [ 2.16386116  4.34404737  0.18887936  0.23038976]
 [-0.0362048   0.18894438  1.63343674  1.84957005]
 [-0.02407

-1702.7885699831477
model -- 0 new_A: 
[[ 1.00050667e+00  9.30882311e-01  6.61470107e-04 -4.64035574e-02]
 [ 1.99864400e-04  9.65570518e-01  4.72645272e-04 -3.97377400e-02]
 [-2.65999644e-04 -2.91458783e-02  1.00031445e+00  9.31936859e-01]
 [-2.74616054e-04 -2.58772358e-03 -3.82130692e-05  9.81917396e-01]]
model -- 1 new_A: 
[[ 1.00050667e+00  9.30882313e-01  6.61470075e-04 -4.64035676e-02]
 [ 1.99864264e-04  9.65570520e-01  4.72645247e-04 -3.97377474e-02]
 [-2.65999537e-04 -2.91458784e-02  1.00031445e+00  9.31936867e-01]
 [-2.74615969e-04 -2.58772474e-03 -3.82130684e-05  9.81917403e-01]]
model -- 0 new_Q: 
[[ 1.89263933  2.16255592 -0.03667413 -0.02385341]
 [ 2.16255741  4.34321289  0.18950423  0.23150728]
 [-0.03655258  0.18956943  1.63600769  1.84891795]
 [-0.02378376  0.23159746  1.84891734  3.69174022]]
model -- 1 new_Q: 
[[ 1.89263941  2.16255593 -0.03667421 -0.02385346]
 [ 2.16255741  4.3432129   0.18950419  0.23150722]
 [-0.03655267  0.18956939  1.63600777  1.84891803]
 [-0.023

Filter model. Testing with learnt parameters from EMGPB2.

In [8]:
em_transition_model_1 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[0].A,
                                                                   covariance_matrix=new_models[0].Q)
em_transition_model_2 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[1].A,
                                                                   covariance_matrix=new_models[1].Q)
em_measurement_noise_1 = new_models[0].R
em_measurement_noise_2 = new_models[1].R

em_measurement_model_1 = LinearGaussian(ndim_state=4, mapping=[0, 2],
                                   noise_covar=em_measurement_noise_1)
em_measurement_model_2 = LinearGaussian(ndim_state=4, mapping=[0, 2],
                                   noise_covar=em_measurement_noise_2)

em_model_transition_matrix = np.array([[0.5, 0.5],
                                       [0.5, 0.5]])
em_predictor_1 = KalmanPredictor(em_transition_model_1)
em_predictor_2 = KalmanPredictor(em_transition_model_2)
em_imm_predictor = IMMPredictor([em_predictor_1, em_predictor_2], em_model_transition_matrix)

em_updater_1 = KalmanUpdater(em_measurement_model_1)
em_updater_2 = KalmanUpdater(em_measurement_model_2)
em_imm_updater = IMMUpdater([em_updater_1, em_updater_2], em_model_transition_matrix)

em_state_init = WeightedGaussianState(StateVector([[0], [0], [0], [0]]),
                                      CovarianceMatrix(np.diag([5.0, 5.0, 5.0, 5.0])),
                                      timestamp=timestamp_init,
                                      weight=0.5)

prior = GaussianMixtureState([copy(em_state_init), copy(em_state_init)])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)-1):
    # measurement
    measurement = detection_track[i]
    # State prediction
    prediction = em_imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = em_imm_updater.predict_measurement(prediction)
    # Association
    hyp = SingleHypothesis(prediction, measurement)
    # State update
    prior = em_imm_updater.update(hyp)
    track.append(prior)
    track_error.append(em_measurement_model_1.function(track[i].state_vector, np.zeros((2,1))) - em_measurement_model_1.function(gt_track[i].state_vector, np.zeros((2,1))))

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 0.5783371153888026
